# Temperature, Salinity, and Stratification #

In [5]:
import xarray as xr
from matplotlib import pyplot as plt
import seaborn
import numpy as np
import gsw
seaborn.set(font_scale=2)
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,7)

In [6]:
# masks
woa_mask = xr.open_dataset('http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NODC/.WOA09/'
                             '.Masks/.basin/dods/').rename({'X':'lon', 'Y':'lat', 'Z': 'depth'})
basin_names = woa_mask.basin.attrs['CLIST'].split('\n')
basins_main = { n: basin_names[n-1] for n in [1,2,3,10] }
basins_marginal = { n: basin_names[n-1] for n in [4,5,6,7,11] }

In [7]:
woa = xr.open_dataset('http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NODC/.WOA09/'
                        '.Grid-1x1/.Annual/dods')
woa = xr.merge(woa_mask)

RuntimeError: NetCDF: file not found

In [ ]:
fig, ax = plt.subplots()
woa['temperature.t_an'][0,0].plot(yincrease=True, vmax=30, center=14, ax=ax)
ax.set_title('Sea Surface Temperatre')

In [ ]:
fig, ax = plt.subplots()
woa['temperature.t_an'][0].mean(dim='lon').plot.contourf(
    yincrease=False, vmax=30, center=14, levels=np.arange(-2,30))
ax.set_title('Zonal Mean Temperature')

In [ ]:
fig, ax = plt.subplots()
woa['salinity.s_an'][0,0].plot(yincrease=True, vmax=37, vmin=32)
ax.set_title('Sea Surface Salinity')

In [ ]:
fig, ax = plt.subplots()
woa['salinity.s_an'][0].mean(dim='lon').plot.contourf(
    yincrease=False, levels=np.arange(33,36,0.2))
ax.set_title('Zonal Mean Salinity')

In [ ]:
woa['thick'] = woa.depth.diff(dim='depth')
# add an area element
woa['area'] = (gsw.earth.earth_radius**2 *
               np.radians(1.) * np.radians(1.) *
               np.cos(np.radians(woa.lat)))
t, s, dz, a, basin = xray.broadcast_arrays(
                *xray.align(
                    woa['temperature.t_an'], woa['salinity.s_an'],
                    woa['thick'], woa['area'], woa['basin']
                )
              )
vol = dz*a

In [ ]:
fig, (ax1, ax2) = plt.subplots(2)

t.plot.hist(bins=100, ax=ax1,
            normed=True, range=(-2,30),
            weights=np.ma.masked_invalid(vol.where(~np.isnan(t)).values).compressed());

s.plot.hist(bins=100, ax=ax2,
            normed=True, range=(30,40),
            weights=np.ma.masked_invalid(vol.where(~np.isnan(s)).values).compressed());

In [ ]:
fig

In [ ]:
from matplotlib.colors import LogNorm
plt.hexbin(s.values.ravel(), t.values.ravel(),
           C=vol.values.ravel(), reduce_C_function=np.sum,
           extent=(30,40,-3,30), gridsize=50, bins='log',
           cmap='RdYlBu_r')
plt.colorbar()
plt.clim([12,18])

In [ ]:
pal = seaborn.palettes.color_palette(n_colors=len(basins_main))
fig, ax = plt.subplots()
for (bnum, bname), col in zip(basins_main.iteritems(), pal):
    ax.plot(s.where(basin==bnum).values.ravel(), t.where(basin==bnum).values.ravel(),
             '.', color=col,
                alpha=0.1, label=bname
               )
ax.set_xlim(30,38)
ax.legend(loc='upper left')



In [ ]:
fig

In [ ]:
# the goods are here
# https://www.nodc.noaa.gov/OC5/3M_HEAT_CONTENT/
# http://data.nodc.noaa.gov/thredds/catalog/woa/heat_content/heat_content/catalog.html
hc = xray.open_dataset('http://data.nodc.noaa.gov/thredds/dodsC/woa/'
                       'heat_content/heat_content/heat_content_anomaly_0-700_pentad.nc',
                        decode_times=False)
# fix time to something xray likes
hc.time *= (365/12.)
tunits = hc.time.attrs['units'].replace('months', 'days')
hc.time.attrs['units'] = tunits
hc = xray.decode_cf(hc)

In [ ]:
fig, ax = plt.subplots()
hc['h18_hc'].mean(dim=('lon','lat','depth')).plot(ax=ax)
ax.set_ylabel(hc.h18_hc.attrs['units'])
plt.title('Ocean Heat Content')